In [11]:
import os
import h5py
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from torchvision import transforms
from torchvision.models import resnet50, ResNet50_Weights
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, f1_score
import torch.nn.functional as F
import functools
from torch.optim import Adam
import torchvision.transforms as transforms
import os
import random
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, recall_score, precision_score, confusion_matrix


In [ ]:
import h5py
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from tqdm import tqdm
import numpy as np
import random
import timm
from timm.scheduler.cosine_lr import CosineLRScheduler
import torch.backends.cudnn as cudnn
from torchsampler import ImbalancedDatasetSampler
import os
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, recall_score, precision_score, confusion_matrix

# --- Mixup関数 ---
def mixup_data(x, y, alpha=1.0):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(x.device)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

class EarlyStopping:
    def __init__(self, patience=5, verbose=False):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def __call__(self, acc):
        score = acc
        if self.best_score is None or score > self.best_score:
            self.best_score = score
            self.counter = 0
        else:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True

class PCamDataset(Dataset):
    def __init__(self, h5_x_path, h5_y_path=None, transform=None):
        self.x_path = h5_x_path
        self.y_path = h5_y_path
        self.transform = transform
        self.has_labels = h5_y_path is not None

        self.x_file = h5py.File(h5_x_path, 'r')
        self.length = len(self.x_file['x'])

        if self.has_labels:
            self.y_file = h5py.File(h5_y_path, 'r')
            self.labels = self.y_file['y'][:]

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        image = self.x_file['x'][idx].astype(np.uint8)
        image = transforms.ToPILImage()(image)
        if self.transform:
            image = self.transform(image)
        if self.has_labels:
            label = self.labels[idx].item() 
            return image, label
        else:
            return image

    def __del__(self):
        if hasattr(self, 'x_file'):
            self.x_file.close()
        if self.has_labels and hasattr(self, 'y_file'):
            self.y_file.close()

    def get_labels(self):
        if self.has_labels:
            return self.labels.reshape(-1)  # 1次元に変換
        else:
            return None

# RandAugment + (0.5,0.5,0.5)正規化
mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]

train_transform = transforms.Compose([
    transforms.Resize((96, 96)),
    transforms.RandAugment(num_ops=2, magnitude=14),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

eval_transform = transforms.Compose([
    transforms.Resize(96),
    transforms.CenterCrop(96),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

train_dataset = PCamDataset(
    '/home/gotou/Medical/b4us/pcamdata/camelyonpatch_level_2_split_train_x.h5',
    '/home/gotou/Medical/b4us/pcamdata/camelyonpatch_level_2_split_train_y.h5',
    transform=train_transform
)
train_loader = DataLoader(train_dataset, batch_size=64, sampler=ImbalancedDatasetSampler(train_dataset), num_workers=4)

val_dataset = PCamDataset(
    '/home/gotou/Medical/b4us/pcamdata/valid_x_uncompressed.h5',
    '/home/gotou/Medical/b4us/pcamdata/valid_y_uncompressed.h5',
    transform=eval_transform
)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4)

torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
torch.cuda.manual_seed_all(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# --- モデル選択 ---
model_name = 'resnet50'  # 例: 'resnet50', 'efficientnet_b1', 'vit_base_patch16_224' など
num_classes = 2
model = timm.create_model(model_name, pretrained=True, num_classes=num_classes)

if torch.cuda.is_available() and torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs!")
    model = nn.DataParallel(model)
model = model.to(device)

# --- Optimizer選択 ---
optimizer_name = 'adam'  # 'adam' or 'sgd'
lr = 1e-4
if optimizer_name == 'adam':
    optimizer = optim.Adam(model.parameters(), lr=lr)
elif optimizer_name == 'sgd':
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
else:
    raise ValueError('Unknown optimizer')

# --- CosineLRScheduler (timm) ウォームアップ付き ---
n_epochs = 50
scheduler = CosineLRScheduler(optimizer, t_initial=n_epochs, lr_min=lr*0.05, warmup_t=int(0.05*n_epochs), warmup_lr_init=lr*0.05, warmup_prefix=True)

criterion = nn.CrossEntropyLoss()

scaler = torch.cuda.amp.GradScaler(enabled=True)

best_acc = 0
train_losses = []
val_accuracies = []

early_stopping = EarlyStopping(patience=7, verbose=True)

for epoch in range(n_epochs):
    model.train()
    total_loss = 0
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{n_epochs}")
    for imgs, labels in loop:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        imgs_mix, targets_a, targets_b, lam = mixup_data(imgs, labels, alpha=0.4)
        with torch.cuda.amp.autocast(enabled=True):
            outputs = model(imgs_mix)
            loss = mixup_criterion(criterion, outputs, targets_a, targets_b, lam)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        total_loss += loss.item()
        loop.set_postfix(loss=loss.item())
    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1} Training Loss: {avg_train_loss:.4f}")

    # --- Validation ---
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            with torch.cuda.amp.autocast(enabled=True):
                outputs = model(imgs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    acc = 100 * correct / total
    print(f"Validation accuracy after epoch {epoch+1}: {acc:.2f}%")
    train_losses.append(avg_train_loss)
    val_accuracies.append(acc)
    if acc > best_acc:
        best_acc = acc
        torch.save(model.state_dict(), 'bestresnet50.pth')
        print("Best model saved.")
    scheduler.step(epoch+1)
    early_stopping(acc)
    if early_stopping.early_stop:
        print("Early stopping!")
        break

model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

/tmp/ipykernel_362939/2211473014.py:159: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=True)


Using 4 GPUs!


Epoch 1/50:   0%|          | 0/4096 [00:00<?, ?it/s]/tmp/ipykernel_362939/2211473014.py:175: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=True):
Epoch 1/50: 100%|██████████| 4096/4096 [22:10<00:00,  3.08it/s, loss=0.608]

Epoch 1 Training Loss: 0.6327


...

Validation accuracy after epoch 1: 78.04%
Best model saved.


Epoch 2/50: 100%|██████████| 4096/4096 [22:05<00:00,  3.09it/s, loss=0.598]

Epoch 2 Training Loss: 0.4981


Validation accuracy after epoch 2: 83.07%
Best model saved.


Epoch 3/50:  79%|███████▉  | 3243/4096 [17:31<04:34,  3.11it/s, loss=0.591]